## 导入相关库和网络

使用 `res32x4` 去蒸馏 `res8x4`

In [1]:
import os
import random
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

# 教师网络和学生网络
from nets.resnet import resnet32x4, resnet8x4
# 知识蒸馏 KD 的损失函数
from loss.kd import loss

# TensorBoard
Train_Info = "KD : Res32x4 To Res8x4"
writer = SummaryWriter(comment=Train_Info)

/home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 设置随机数种子, 从而可以复现
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(42)

In [3]:
# GPU, 将 '1' 里的数字改为您的设备上的gpu编号
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

## 定义超参数

In [4]:
T = 4               # temperature : 知识蒸馏中的温度
ALPHA = 0.1         # alpha : hard_loss(硬损失交叉熵)的loss weight 
BETA = 0.9          # beta : soft_loss(软损失KL散度)的loss weight
N = 100             # num_classes : 类别数
EPOCH = 20          # epoch : 训练轮数
BATCH_SIZE = 128    # batch_size : 批处理大小 
LR = 0.05           # learning_rate : 初试学习率

# 其余的超参数, 例如 : 优化器中的 momentum, weight_decay, milestones, gamma 等一般情况下很少变动.
# 当EPOCH变化的时候, milestones也要随之变化. 

## 加载教师模型, 以及定义学生网络

In [5]:
res32x4 = resnet32x4(num_classes=N)
ckpt = torch.load("checkpoints/teacher/ckpt_epoch_240.pth", map_location='cpu')
res32x4.load_state_dict(ckpt["model"])
res32x4 = nn.DataParallel(res32x4).cuda()
res32x4.eval()

res8x4 = resnet8x4(num_classes=N)
res8x4 = torch.nn.DataParallel(res8x4).cuda()

teacher_net = res32x4
student_net = res8x4

## 加载数据集
第一次使用会先进行下载, 如果下载的很慢, 可以手动下载数据集然后拖入到 data 文件夹下

In [6]:
# 准备数据集并预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 先四周填充0，在把图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),     # 图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)), #R,G,B每层的归一化用到的均值和方差
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

DATA_PATH = "data"
# 训练数据集
# num_workers一般情况下取决于 cpu 的性能
trainset = torchvision.datasets.CIFAR100(root=DATA_PATH, train=True, download=True, transform=transform_train) 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)   
# 测试数据集
testset = torchvision.datasets.CIFAR100(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)


Files already downloaded and verified
Files already downloaded and verified


## 优化器

一般在各种论文中 cifar-100 数据集上的 `epoch` 都会选择设置为 `240`

其对应的 `milestones`的值为 `[150, 180, 210]`

这里仅做展示, 所有 `epoch` 设置了 `40`,  `milestones`的值为 `[15, 25, 35]`

In [7]:
optimizer = torch.optim.SGD(student_net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25, 35], gamma=0.1)

## Train函数和Test函数

In [8]:
# 分别定义训练集和测试集上的最佳Acc, 使用 global 修饰为全局变量, 然后再训练期间更新
best_train_acc = 0
best_test_acc = 0

In [9]:
from tqdm import tqdm

def train(epoch):
    global best_train_acc

    # 设置学生模型为训练模式
    student_net.train()

    print('\nEpoch: %d' % epoch)

    train_loss = 0
    correct = 0
    total = 0

    # 使用 tqdm 包装 trainloader 以显示进度条
    with tqdm(trainloader, desc=f"Training Epoch {epoch}", total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()

            logits_student, _ = student_net(inputs)
            with torch.no_grad():
                logits_teacher, _ = teacher_net(inputs)

            # 硬损失
            ce_loss = nn.CrossEntropyLoss()(logits_student, targets)
            # 软损失
            kd_loss = loss(logits_student, logits_teacher, temperature=T)
            total_loss = ALPHA * ce_loss + BETA * kd_loss

            total_loss.backward()
            optimizer.step()

            train_loss += total_loss.item()
            _, predicted = logits_student.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # 更新 TensorBoard
            writer.add_scalar('Train/Accuracy', 100. * correct / total, batch_idx + (epoch - 1) * 782)
            writer.add_scalar('Train/Loss', total_loss.item(), batch_idx + (epoch - 1) * 782)

            # 使用 set_postfix 更新进度条的后缀
            pbar.set_postfix(loss=train_loss / (batch_idx + 1), acc=f"{100. * correct / total:.1f}%")

    # 如果当前训练集上的准确率高于 best_test_acc，则更新 best_test_acc
    acc = 100 * correct / total
    if acc > best_train_acc:
        best_train_acc = acc


In [10]:
def test(net, epoch):
    global best_test_acc
    net.eval()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        # 使用 tqdm 包装 testloader 以显示进度条
        with tqdm(testloader, desc=f"Testing Epoch {epoch}", total=len(testloader)) as pbar:
            for batch_idx, (inputs, targets) in enumerate(pbar):

                inputs, targets = inputs.cuda(), targets.cuda()
                logits_student, _ = net(inputs)

                loss = nn.CrossEntropyLoss()(logits_student, targets)

                test_loss += loss.item()
                _, predicted = logits_student.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                # 在 tqdm 进度条的后缀中显示当前损失和准确率
                pbar.set_postfix(loss=test_loss / (batch_idx + 1), acc=f"{100. * correct / total:.1f}%")

                # 更新 TensorBoard
                writer.add_scalar('Test/Accuracy', 100. * correct / total, batch_idx + (epoch - 1) * 157)
                writer.add_scalar('Test/Loss', loss.item(), batch_idx + (epoch - 1) * 157)

        # 计算当前测试集上的准确率
        acc = 100. * correct / total

        # 如果当前测试集上的准确率高于 best_test_acc，则更新 best_test_acc
        # 并且将学生模型保存下来
        if acc > best_test_acc:
            print('Saving..')
            torch.save(student_net, 'checkpoints/student/kd_res8x4.pth')
            best_test_acc = acc

## 训练

In [11]:
for epoch in range(1, EPOCH + 1) :
    train(epoch)
    test(student_net, epoch)

    # 更新学习率
    scheduler.step()



Epoch: 1


Testing Epoch 1: 100%|██████████| 79/79 [00:06<00:00, 13.00it/s, acc=14.0%, loss=3.78]


Saving..

Epoch: 2


Testing Epoch 2: 100%|██████████| 79/79 [00:06<00:00, 12.45it/s, acc=18.0%, loss=4.01]


Saving..

Epoch: 3


...

Saving..

Epoch: 19


Testing Epoch 19: 100%|██████████| 79/79 [00:06<00:00, 12.95it/s, acc=66.2%, loss=1.33]


Saving..

Epoch: 20


Testing Epoch 20: 100%|██████████| 79/79 [00:06<00:00, 11.53it/s, acc=66.3%, loss=1.34]

Saving..


In [12]:
print(Train_Info)
print('best_Train_Acc = ', best_train_acc)
print('best_Test_Acc = ', best_test_acc)

KD : Res32x4 To Res8x4
best_Train_Acc =  73.41
best_Test_Acc =  66.31


In [13]:
# 启动 tensorboard

TensorFlow installation not found - running with reduced feature set.
/home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.29' not found (required by /home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by /home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/huasi/miniconda3/envs/dkd/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.